In [1]:
import tensorflow as tf
import numpy as np
import random
from collections import deque

# --- Model creation ---
def create_cnn_dqn(action_size=4):
    grid_input = tf.keras.Input(shape=(20, 20, 1), name='grid_input')
    x1 = tf.keras.layers.Conv2D(16, 3, activation='tanh', padding='same')(grid_input)
    x1 = tf.keras.layers.MaxPooling2D(2)(x1)
    x1 = tf.keras.layers.Flatten()(x1)

    pose_input = tf.keras.Input(shape=(4,), name='pose_input')
    x2 = tf.keras.layers.Dense(32, activation='relu')(pose_input)

    x = tf.keras.layers.Concatenate()([x1, x2])
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    output = tf.keras.layers.Dense(action_size, activation='linear')(x)

    model = tf.keras.Model(inputs=[grid_input, pose_input], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse')
    return model

# --- Hyperparameters ---
ACTION_SIZE = 4
EPISODES = 1000
EPSILON = 1.0
EPSILON_DECAY = 0.995
EPSILON_MIN = 0.05
GAMMA = 0.99
BATCH_SIZE = 32
MEMORY_SIZE = 10000
TARGET_UPDATE_FREQ = 10

# --- Initialize networks and memory ---
policy_net = create_cnn_dqn(ACTION_SIZE)
target_net = create_cnn_dqn(ACTION_SIZE)
target_net.set_weights(policy_net.get_weights())

dummy_grid = np.zeros((1, 20, 20, 1), dtype=np.float32)
dummy_pose = np.zeros((1, 4), dtype=np.float32)
policy_net.predict([dummy_grid, dummy_pose])
target_net.predict([dummy_grid, dummy_pose])
memory = deque(maxlen=MEMORY_SIZE)


2025-07-17 12:47:40.427660: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-17 12:47:40.428643: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 12:47:40.431096: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 12:47:40.438591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752736660.449953  685381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752736660.45

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [2]:
import numpy as np

def get_local_map(global_map, robot_pose, size=20):
    half = size // 2
    r, c = robot_pose
    local_map = np.full((size, size), 1, dtype=np.int8)  # initialize with padding value 1

    for i in range(-half, half):
        for j in range(-half, half):
            gi, gj = r + i, c + j
            li, lj = i + half, j + half  # local map indices
            if 0 <= gi < global_map.shape[0] and 0 <= gj < global_map.shape[1]:
                local_map[li, lj] = global_map[gi, gj]
            # else: keep the padding value = 1

    return local_map


def safe_move(robot_pose, action_map,action, rows, cols):
    flag = True

    while flag:
      move = action_map[action]
      new_x = robot_pose[0] + move[0]
      new_y = robot_pose[1] + move[1]
      if 0 <= new_x < rows and 0 <= new_y < cols:
          flag = False
          return [new_x, new_y]
      else:
        action = random.randint(0,3)  # or return unchanged

In [3]:
def compute_reward(robot_pose, goal_pose, action,collided, reached_goal, previous_distance, current_distance, explored_cells,visited_set,occ_map,recent_pos):
    reward = 0.0
    #print(previous_distance,current_distance)
    if recent_pos.count(tuple(robot_pose)) >=1:
        reward = reward - 20* recent_pos.count(tuple(robot_pose))
    # if tuple(robot_pose) in visited_set:
    #     reward = reward -30;
    # else:
    #     visited_set.add(tuple(robot_pose))
    #     reward = reward +10;
    if robot_pose[0] == goal_pose[0]:
        reward = reward +200;
        if (goal_pose[1]-robot_pose[1] > 0):
            if (action[1] == 1):
                reward += 200;
            else:
                reward -= 100;
        if (goal_pose[1]-robot_pose[1] < 0):
            if (action[1] == -1):
                reward += 200;
            else:
                reward -= 100;
    if robot_pose[1] == goal_pose[1]:
        reward = reward +200;
        if (goal_pose[0]-robot_pose[0] > 0):
            if (action[0] == 1):
                reward += 200;
            else:
                reward -= 100;
        if (goal_pose[0]-robot_pose[0] < 0):
            if (action[0] == -1):
                reward += 200;
            else:
                reward -= 100;
    if collided:
        return reward -1000.0;  # Large negative for hitting wall or obstacle

    if reached_goal:
        return reward +1000.0; # Goal reached

    if (57< robot_pose[0] | robot_pose[0]<3 ):
        reward = reward-20;
    else:
        reward= reward +10;
    if (57< robot_pose[1] | robot_pose[1]< 3):
        reward = reward -20;
    else:
        reward = reward +10;
    # Reward getting closer to goal
    if current_distance < previous_distance:
        reward += 20.0
    else:
        reward -= 20.0  # Penalize going away or staying same

    # Small penalty to discourage taking too long
    reward -= 5
    reward = reward + 60 - np.abs(goal_pose[0]-robot_pose[0])
    reward = reward + 60 - np.abs(goal_pose[1]-robot_pose[1])

    # Bonus for exploring new cells in occupancy grid
    reward += 0.1 * explored_cells  # Optional if tracking exploration

    dx, dy = action
    isclear = True
    for step in range(1, 11):  # Look 10 cells ahead
        nx, ny = robot_pose[0] + dx * step, robot_pose[1] + dy * step
        if 0 <= nx < occ_map.shape[0] and 0 <= ny < occ_map.shape[1]:
            if occ_map[nx, ny] >= 1:  # Obstacle detected
                reward -= 10*(10- step)  # Penalty for heading toward obstacle
                isclear= False
                break  # Stop after first obstacle detected
        else:
            reward -= 5  # Penalty for moving out of bounds
            break
    if(isclear):
        reward = reward+5
    

    return reward

In [4]:
def env_step(state, action, goal, direction, huristic, explored, recent_pos):
    """
    Simulates one environment step.
    
    Parameters:
        state          : global map (2D numpy array)
        action         : new robot position [x, y]
        goal           : goal position [x, y]
        direction      : movement direction (dx, dy)
        huristic       : previous distance to goal
        explored       : set of visited positions
        recent_pos     : deque of recent positions
    
    Returns:
        next_state     : new 20x20 local map (1, 20, 20, 1)
        reward         : reward for the step
        done           : whether the episode should terminate
    """
    # Generate new local map from global state
    next_map = get_local_map(state, action, size=20)
    next_state = next_map.astype(np.float32).reshape(1, 20, 20, 1)

    # Check for collision
    collided = state[action[0], action[1]] >= 1

    # Check if goal is reached
    reached_goal = (action[0] == goal[0]) and (action[1] == goal[1])

    # Compute distance to goal
    previous_distance = huristic
    current_distance = np.sqrt(
        np.square(goal[0] - action[0]) + np.square(goal[1] - action[1])
    )

    # Compute reward (you must define this function separately)
    # reward = compute_reward(
    #     action, goal, direction, collided, reached_goal,
    #     previous_distance, current_distance,
    #     10, explored=explored,
    #     state=state, recent_pos=recent_pos
    # )
    reward = compute_reward(action, goal,direction, collided, reached_goal, previous_distance, current_distance, 10,explored,state,recent_pos)

    # Done if collided or reached goal
    done = reached_goal or collided

    return next_state, reward, done


In [ ]:
from collections import deque

MAX_STEPS_EP = 3600

global_map = np.zeros((60, 60), dtype=np.int8)
global_map[10:15, 10:50] = 1
global_map[30:35, 5:55] = 1
global_map[41:42, 41:42 ] = 1

explored = set()

robot_pose = [5, 5]
goal_pose = [40, 40]

# Define exactly 10 robot–goal pairs (manually indexed)
robot_goal_map = {
    0: ((5, 5), (8, 55)),
    1: ((5, 5), (50,3)),
    2: ((5, 5), (25, 25)),
    3: ((8, 8), (38, 42)),
    4: ((5, 55), (50, 20)),
    5: ((25, 25), (5, 5)),
    6: ((8, 30), (45, 10)),
    7: ((55, 55), (20, 10)),
    8: ((50, 30), (5, 30)),
    9: ((3, 40), (40, 3)),
}


action_map = {
    0: (-1, 0),  # up
    1: (1, 0),   # down
    2: (0, -1),  # left
    3: (0, 1) # right
}

for episode in range(EPISODES):
    # --- Reset environment ---
    #print(robot_pose)
    pair_id = episode // 100
    robot_pose, goal_pose = robot_goal_map[pair_id]
    print("pair id: " + str(pair_id) + " robo: " + str(robot_pose) + " goal: " + str(goal_pose))
    robot_pose = list(robot_pose)
    goal_pose = list(goal_pose)
    recent_pos = deque(maxlen=20)
    local_map = get_local_map(global_map, robot_pose, size=20) # np.random.randint(0, 2, (20, 20, 1)).astype(np.float32)  # Example
    #print(local_map.shape)
    state_grid = local_map.astype(np.float32).reshape(1, 20, 20, 1)
   # state_grid = local_map.astype(np.float32).transpose(0, 2, 3, 1)
    state_pose = np.array([[robot_pose[0], robot_pose[1], goal_pose[0], goal_pose[1]]], dtype=np.float32)             # Example
    done = False
    total_reward = 0
    #print("rbpose",robot_pose)
    step_count = 0
    while not done and step_count<=MAX_STEPS_EP:
        step_count += 1
        # --- Epsilon-greedy action selection ---
        # if np.random.rand() <= EPSILON:
        #     action = random.randint(0, ACTION_SIZE - 1)
        # else:
        #     q_vals = policy_net.predict([state_grid[np.newaxis, ...], state_pose[np.newaxis, ...]], verbose=0)
        #     action = np.argmax(q_vals[0])

        # --- Take action in environment ---
        #print("current", robot_pose)
        huristic = np.sqrt(np.square(goal_pose[0] - robot_pose[0]) + np.square(goal_pose[1] - robot_pose[1]))
        q_values = policy_net.predict([state_grid, state_pose], verbose=0)
        action = np.argmax(q_values[0])
        move = action_map[action]
       # print(move)
       # robot_pose[0] = robot_pose[0] + move[0]
       # robot_pose[1] = robot_pose[1] + move[1]
        robot_pose = safe_move(robot_pose,action_map,action,60,60)
        recent_pos.append(tuple(robot_pose))
        next_grid, reward, done = env_step(global_map, robot_pose,goal_pose,move,huristic,explored,recent_pos)  # Placeholder
        next_pose = np.array([[robot_pose[0], robot_pose[1], goal_pose[0], goal_pose[1]]], dtype=np.float32)
        #print(robot_pose,reward)
        memory.append((state_grid, state_pose, action, reward, next_grid, next_pose, done))

        state_grid, state_pose = next_grid, next_pose
        total_reward += reward

        # --- Train if enough samples ---
        if len(memory) >= BATCH_SIZE:
            minibatch = random.sample(memory, BATCH_SIZE)

            grids = np.array([x[0] for x in minibatch])
            poses = np.array([x[1] for x in minibatch])
            actions = np.array([x[2] for x in minibatch])
            rewards = np.array([x[3] for x in minibatch])
            next_grids = np.array([x[4] for x in minibatch])
            next_poses = np.array([x[5] for x in minibatch])
            dones = np.array([x[6] for x in minibatch])

          #  print("grids shape:", grids.shape)   # Should be (batch_size, 20, 20, 1)
           # print("poses shape:", poses.shape)   # Should be (batch_size, 4)

            q_targets = policy_net.predict([grids.reshape(-1,20,20,1), poses.reshape(-1,4)], verbose=0)
            q_next = target_net.predict([next_grids.reshape(-1,20,20,1), next_poses.reshape(-1,4)], verbose=0)

            for i in range(BATCH_SIZE):
                target = rewards[i]
                if not dones[i]:
                    target += GAMMA * np.amax(q_next[i])
                q_targets[i][actions[i]] = target

            # --- Train policy network ---
            policy_net.fit([grids.reshape(-1,20,20,1), poses.reshape(-1,4)], q_targets, verbose=0)

    # --- Update target network ---
    if episode % TARGET_UPDATE_FREQ == 0:
        target_net.set_weights(policy_net.get_weights())

    # --- Epsilon decay ---
    if EPSILON > EPSILON_MIN:
        EPSILON *= EPSILON_DECAY
    if episode % 50 == 0:
        policy_net.save(f"/storage/projects2/e19-4yp-g28-peraswarm-local-nav/checkpoint_ep{episode}.h5")
        print(f" Saved checkpoint at episode {episode}")
    print(f"Episode {episode+1}/{EPISODES} - Total reward: {total_reward:.2f} - Epsilon: {EPSILON:.2f}")
    
policy_net.save("path_planing_model_final.h5")


pair id: 0 robo: (5, 5) goal: (8, 55)


 Saved checkpoint at episode 0
Episode 1/1000 - Total reward: -310.00 - Epsilon: 0.99
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 2/1000 - Total reward: -310.00 - Epsilon: 0.99
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 3/1000 - Total reward: 13411.00 - Epsilon: 0.99
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 4/1000 - Total reward: 37368.00 - Epsilon: 0.98
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 5/1000 - Total reward: 224551.00 - Epsilon: 0.98
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 6/1000 - Total reward: 225489.00 - Epsilon: 0.97
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 7/1000 - Total reward: 281601.00 - Epsilon: 0.97
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 8/1000 - Total reward: 26609.00 - Epsilon: 0.96
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 9/1000 - Total reward: 26609.00 - Epsilon: 0.96
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 10/1000 - Total reward: 26609.00 - Epsilon: 0.95
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 11/1000 - Total rew

 Saved checkpoint at episode 50
Episode 51/1000 - Total reward: 27009.00 - Epsilon: 0.77
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 52/1000 - Total reward: 27009.00 - Epsilon: 0.77
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 53/1000 - Total reward: 27009.00 - Epsilon: 0.77
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 54/1000 - Total reward: 27009.00 - Epsilon: 0.76
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 55/1000 - Total reward: 27009.00 - Epsilon: 0.76
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 56/1000 - Total reward: 27009.00 - Epsilon: 0.76
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 57/1000 - Total reward: 27009.00 - Epsilon: 0.75
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 58/1000 - Total reward: 27009.00 - Epsilon: 0.75
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 59/1000 - Total reward: 27009.00 - Epsilon: 0.74
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 60/1000 - Total reward: 27009.00 - Epsilon: 0.74
pair id: 0 robo: (5, 5) goal: (8, 55)
Episode 61/1000 - 